# TP 4 - Création de panoramiques

##### Presser une touche pour continuer & esc pour arreter

In [2]:
import cv2
import numpy as np

ESC_KEY = 27

class MyFeatureDetector:
    @staticmethod
    def createORB():
        return cv2.ORB_create(10000)  # Paramètre ajustable: Nombre maximum de points-clés à détecter

class MyDescriptorExtractor:
    @staticmethod
    def createORB():
        return cv2.ORB_create(10000)  # Paramètre ajustable: Nombre maximum de points-clés à détecter

class MyImageProcessor:
    def __init__(self, image_path):
        self._myImage = cv2.imread(image_path)
        if self._myImage is None or self._myImage.size == 0 or self._myImage.shape[0] == 0 or self._myImage.shape[1] == 0:
            print(f"Ne parvient pas à ouvrir {image_path}")
            exit(1)

    def displayImage(self, window_name):
        cv2.namedWindow(window_name)
        cv2.imshow(window_name, self._myImage)

    def displayKeypoints(self, keypoints, rgb=(0, 0, 255)):
        if keypoints:
            img_with_keypoints = cv2.drawKeypoints(self._myImage, keypoints, None, color=rgb)
            cv2.imshow("Image avec points clés", img_with_keypoints)
            return img_with_keypoints

def main():
    # Charger les images
    image_processor1 = MyImageProcessor("img/IMG_3_reduced.jpg")
    image_processor2 = MyImageProcessor("img/IMG_4_reduced.jpg")

    # Afficher les images
    image_processor1.displayImage("Image 1")
    image_processor2.displayImage("Image 2")

    # Détection et calcul des caractéristiques pour l'image 1
    feature_detector = MyFeatureDetector.createORB()
    keypoints1, descriptors1 = feature_detector.detectAndCompute(image_processor1._myImage, None)
    image_processor1.displayKeypoints(keypoints1)

    # Attendre une touche pour continuer
    cv2.waitKey(0)

    # Détection et calcul des caractéristiques pour l'image 2
    keypoints2, descriptors2 = feature_detector.detectAndCompute(image_processor2._myImage, None)
    image_processor2.displayKeypoints(keypoints2)

    # Attendre une touche pour continuer
    cv2.waitKey(0)

    # Correspondance des descripteurs entre les deux images
    descriptor_extractor = MyDescriptorExtractor.createORB()
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Afficher le nombre total de correspondances
    print(f"Nombre total de correspondances : {len(matches)}")

    # Afficher le nombre de meilleures correspondances
    num_best_matches = 10  # Paramètre ajustable: Nombre de meilleures correspondances à afficher
    print(f"Nombre de meilleures correspondances : {num_best_matches}")

    # Afficher les meilleures correspondances
    for i in range(num_best_matches):
        print(f"Meilleure correspondance {i + 1} : Distance = {matches[i].distance}")

    # Afficher le matching entre les points des images 1 et 2
    match_img = cv2.drawMatches(image_processor1._myImage, keypoints1, image_processor2._myImage, keypoints2, matches[:num_best_matches], None)
    cv2.imshow("Matching", match_img)

    # Calculer l'homographie
    im1Pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    im2Pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    homography, _ = cv2.findHomography(im2Pts, im1Pts, cv2.RANSAC, 1.5)

    if homography is not None:
        # Appliquer l'homographie
        result = cv2.warpPerspective(image_processor2._myImage, homography, (image_processor1._myImage.shape[1] + image_processor2._myImage.shape[1], image_processor2._myImage.shape[0]))

        # Ajouter l'image de gauche
        result[0:image_processor1._myImage.shape[0], 0:image_processor1._myImage.shape[1]] = image_processor1._myImage

        cv2.imshow("Warping", result)

    # Lancer la boucle infinie
    retK = 0
    while retK != ESC_KEY:
        # Attendre une pression de touche
        retK = cv2.waitKey(1)

    # Fin du programme
    cv2.destroyAllWindows()

# Appeler la fonction principale
if __name__ == "__main__":
    main()


Nombre total de correspondances : 3247
Nombre de meilleures correspondances : 10
Meilleure correspondance 1 : Distance = 6.0
Meilleure correspondance 2 : Distance = 10.0
Meilleure correspondance 3 : Distance = 10.0
Meilleure correspondance 4 : Distance = 10.0
Meilleure correspondance 5 : Distance = 11.0
Meilleure correspondance 6 : Distance = 11.0
Meilleure correspondance 7 : Distance = 11.0
Meilleure correspondance 8 : Distance = 11.0
Meilleure correspondance 9 : Distance = 11.0
Meilleure correspondance 10 : Distance = 12.0


### Capture des résultats

![img1 avec points d'interets](./img/IMG_3_reduced.jpg) ![img1 avec points d'interets](./captures/img.png)


![img1 avec points d'interets](./img/IMG_4_reduced.jpg) ![img1 avec points d'interets](./captures/img3.png)
 

![img1 avec points d'interets](./captures/img4.png) ![img1 avec points d'interets](./captures/img5.png)